<a href="https://colab.research.google.com/github/FuadKhalit/Pyspark_Repo/blob/main/Recommendation_System(ALS%20Algorithm).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install pyspark


     |████████████████████████████████| 204.2MB 41kB/s 
     |████████████████████████████████| 204kB 43.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=496aed58d01acd472acc5b8aae25eec115beac127f840eabbcc282f4159a3236
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc, col, max
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder


# **Creating Spark Session**

In [7]:
session1 = SparkSession.builder.appName("Spotify").getOrCreate()

# **Loading Dataset**

In [9]:
file_path = '/content/drive/MyDrive/listenings.csv'
df_listenings = session1.read.format('csv').option('header', True).option('inferSchema', True).load(file_path)
df_listenings.show()

+-----------+-------------+--------------------+---------------+--------------------+
|    user_id|         date|               track|         artist|               album|
+-----------+-------------+--------------------+---------------+--------------------+
|000Silenced|1299680100000|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|1299679920000|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|1299679440000|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|1299679200000|            Acapella|          Kelis|            Acapella|
|000Silenced|1299675660000|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|1297511400000|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|1294498440000|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|1292438340000|               ObZen|      Meshuggah|               ObZen|
|000Silenced|1292437740000|   Yama's Messengers|      

# **Data Pre-Processing**

In [10]:
# remove the date column
df_listenings = df_listenings.drop('date')
df_listenings.show()

+-----------+--------------------+---------------+--------------------+
|    user_id|               track|         artist|               album|
+-----------+--------------------+---------------+--------------------+
|000Silenced|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|            Acapella|          Kelis|            Acapella|
|000Silenced|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|               ObZen|      Meshuggah|               ObZen|
|000Silenced|   Yama's Messengers|         Gojira|The Way of All Flesh|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For No...|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For

In [11]:
df_listenings = df_listenings.na.drop()
df_listenings.show()

+-----------+--------------------+---------------+--------------------+
|    user_id|               track|         artist|               album|
+-----------+--------------------+---------------+--------------------+
|000Silenced|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|            Acapella|          Kelis|            Acapella|
|000Silenced|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|               ObZen|      Meshuggah|               ObZen|
|000Silenced|   Yama's Messengers|         Gojira|The Way of All Flesh|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For No...|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For

In [13]:
total_row = df_listenings.count()
total_column = len(df_listenings.columns)
print(total_row,total_column)

13758905 4


# **Data Exploration**

In [15]:
#perform sql query to dataset

df_listenings_agg = df_listenings.select('user_id','track').groupby('user_id', 'track').agg(count('*').alias('count')).orderBy('user_id')
df_listenings_agg.show()

+-------+--------------------+-----+
|user_id|               track|count|
+-------+--------------------+-----+
| --Seph|Chelsea Hotel - L...|    1|
| --Seph|               Leloo|    1|
| --Seph|          Paris 2004|    7|
| --Seph| Air on the G String|    1|
| --Seph|        Window Blues|    1|
| --Seph|   Summa for Strings|    1|
| --Seph|         The Embrace|    1|
| --Seph|Hungarian Rhapsod...|    1|
| --Seph|     The Way We Were|    1|
| --Seph|Vestido Estampado...|    1|
| --Seph| White Winter Hymnal|    3|
| --Seph|Belina (Original ...|    1|
| --Seph|Hungarian Dance No 5|    1|
| --Seph|       Phantom Pt II|    1|
| --Seph|              Monday|    1|
| --Seph|  California Waiting|    1|
| --Seph|      Hour for magic|    2|
| --Seph|       Life On Mars?|    1|
| --Seph|Virus (Luke Fair ...|    1|
| --Seph|Airplanes [feat H...|    1|
+-------+--------------------+-----+
only showing top 20 rows



In [19]:
total_rows1 = df_listenings_agg.count()
total_column1 = len(df_listenings_agg.columns)
print(total_rows1,total_column1)


9930128 3


In [20]:
#to limit the amount of data to be process to save some processing time
df_listenings_agg = df_listenings_agg.limit (20000)

# **Data Transformation**

In [21]:
#convert user_id & track from string to numeric value
indexing = [StringIndexer(inputCol=col, outputCol=col+'_index').fit(df_listenings_agg)for col in list(set(df_listenings_agg.columns) - set(['count']))]

pipeline = Pipeline(stages=indexing)

data = pipeline.fit(df_listenings_agg).transform(df_listenings_agg)

data.show()

+-------+--------------------+-----+-------------+-----------+
|user_id|               track|count|user_id_index|track_index|
+-------+--------------------+-----+-------------+-----------+
| --Seph| White Winter Hymnal|    3|         69.0|       59.0|
| --Seph|Virus (Luke Fair ...|    1|         69.0|    15896.0|
| --Seph|Airplanes [feat H...|    1|         69.0|      519.0|
| --Seph|Belina (Original ...|    1|         69.0|     3278.0|
| --Seph|              Monday|    1|         69.0|      334.0|
| --Seph|Hungarian Dance No 5|    1|         69.0|     7558.0|
| --Seph|       Life On Mars?|    1|         69.0|     1161.0|
| --Seph|  California Waiting|    1|         69.0|      197.0|
| --Seph|       Phantom Pt II|    1|         69.0|     1377.0|
| --Seph|   Summa for Strings|    1|         69.0|    13739.0|
| --Seph|      Hour for magic|    2|         69.0|     7495.0|
| --Seph|Hungarian Rhapsod...|    1|         69.0|     7559.0|
| --Seph|     The Way We Were|    1|         69.0|    1

In [22]:
#select only 3 column from the data 

data = data.select('user_id_index' ,'track_index', 'count').orderBy('user_id_index')
data.show()

+-------------+-----------+-----+
|user_id_index|track_index|count|
+-------------+-----------+-----+
|          0.0|    10943.0|    1|
|          0.0|    11628.0|    2|
|          0.0|     1349.0|    1|
|          0.0|      381.0|    1|
|          0.0|     8692.0|    1|
|          0.0|     6899.0|    1|
|          0.0|    14044.0|    1|
|          0.0|    15513.0|    1|
|          0.0|    11978.0|    2|
|          0.0|    15176.0|    1|
|          0.0|     8305.0|    1|
|          0.0|    13722.0|    1|
|          0.0|    10620.0|    1|
|          0.0|     4424.0|    1|
|          0.0|    16732.0|    1|
|          0.0|    10630.0|    1|
|          0.0|    12169.0|    1|
|          0.0|     4117.0|    1|
|          0.0|    10336.0|    1|
|          0.0|    16829.0|    1|
+-------------+-----------+-----+
only showing top 20 rows



# **Split Train & Test Dataset**

In [24]:
(training,test) = data.randomSplit([0.7,0.3])

# **Generating the Model**

In [25]:
User = 'user_id_index'
Track = 'track_index'
Count = 'count'

als = ALS(maxIter= 10, regParam=0.01, userCol=User, itemCol=Track, ratingCol=Count)
model = als.fit(training)

prediction = model.transform(test)

# **Top 20 Track Recommendation for Each User**

In [26]:
recommend = model.recommendForAllUsers(20)
recommend.show()

+-------------+--------------------+
|user_id_index|     recommendations|
+-------------+--------------------+
|          148|[[460, 9.167991],...|
|           31|[[12192, 7.945960...|
|           85|[[954, 8.807956],...|
|          137|[[16909, 16.62756...|
|           65|[[16969, 9.138924...|
|           53|[[16909, 9.401364...|
|          133|[[14826, 17.99681...|
|           78|[[2379, 8.210358]...|
|          108|[[182, 9.833125],...|
|           34|[[568, 10.319969]...|
|          101|[[2379, 11.978337...|
|          115|[[368, 8.890206],...|
|          126|[[568, 13.922986]...|
|           81|[[182, 10.728386]...|
|           28|[[15430, 8.362229...|
|           76|[[1738, 11.671547...|
|           26|[[16909, 13.12751...|
|           27|[[13402, 6.223134...|
|           44|[[1738, 11.619315...|
|          103|[[16969, 9.738611...|
+-------------+--------------------+
only showing top 20 rows



In [27]:
#shoe the detail for 1 user
#inference = for user id 148, the system recommend track no 460 that have rating 9.16 as first recommendation
#this follow by track no 182 with rating 7.25, track 9973 rating 7.04 and so on
recommend.take(1)

[Row(user_id_index=148, recommendations=[Row(track_index=460, rating=9.167990684509277), Row(track_index=182, rating=7.253549575805664), Row(track_index=9973, rating=7.048877239227295), Row(track_index=16969, rating=6.990200042724609), Row(track_index=12892, rating=5.945867538452148), Row(track_index=357, rating=5.900254249572754), Row(track_index=348, rating=5.835694789886475), Row(track_index=11756, rating=5.405333518981934), Row(track_index=11689, rating=5.405333518981934), Row(track_index=4660, rating=5.405333518981934), Row(track_index=14379, rating=4.432724952697754), Row(track_index=5325, rating=4.382127285003662), Row(track_index=13563, rating=4.216183662414551), Row(track_index=7484, rating=4.135503768920898), Row(track_index=9552, rating=4.135503768920898), Row(track_index=3343, rating=4.117521286010742), Row(track_index=6260, rating=3.8343615531921387), Row(track_index=12399, rating=3.529304027557373), Row(track_index=938, rating=3.5176312923431396), Row(track_index=5194, ra